In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/Practicas_PLN/Competicion/datos/FinancES_test_kaggle.csv", index_col=0)

df.drop(columns="text", inplace=True)

df["label"] = 0
df.head()
df.to_csv("/content/drive/MyDrive/Practicas_PLN/Competicion/resultados/zero_submission.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h1>Procesamiento del Texto</h1>

In [ ]:
from posixpath import pathsep
import os
import sys
import spacy
import spacy.cli
spacy.cli.download("es_core_news_sm")
spacy.cli.download("es_core_news_lg")
spacy.cli.download("es_dep_news_trf")
import es_core_news_sm
import es_core_news_lg
import es_dep_news_trf

from sklearn.cluster import KMeans
from nltk.stem.snowball import SnowballStemmer
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


KeyboardInterrupt: 

In [ ]:
#nlp = es_core_news_sm.load()
nlp = es_core_news_lg.load()
#nlp = spacy.load("es_dep_news_trf")
stemmer = SnowballStemmer("spanish")

df = pd.read_csv("/content/drive/MyDrive/Practicas_PLN/Competicion/datos/FinancES_train_kaggle.csv", index_col=0)
dfTest = pd.read_csv("/content/drive/MyDrive/Practicas_PLN/Competicion/datos/FinancES_test_kaggle.csv", index_col=0)

textoProcesado = []
textoFila = []
textoTestProcesado = []


tag = [valor for valor in df['label']]


for texto in df['text']:
  textoTokenizado = nlp(texto)
  for token in textoTokenizado:
    if not token.is_stop and token.is_alpha:
      textoFila.append(stemmer.stem(token.text.lower()))

  textoProcesado.append(" ".join(textoFila))
  textoFila.clear()


for texto in dfTest['text']:
  textoTokenizado = nlp(texto)
  for token in textoTokenizado:
    if not token.is_stop and token.is_alpha:
      textoFila.append(stemmer.stem(token.text.lower()))

  textoTestProcesado.append(" ".join(textoFila))
  textoFila.clear()

dfTest.drop(columns="text", inplace=True)

# Entrenamiento con SVC

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(textoProcesado)


#model = svm.SVC()

model = svm.SVC(kernel="rbf",C=100,gamma="auto")

model.fit(X, tag)

SVC(C=100, gamma='auto')

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definir los valores para los parámetros que quieres probar
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1,10]}

# Crear un objeto GridSearchCV
grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=5)

# Entrenar el modelo
grid_search.fit(X, tag)



# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

Mejores hiperparámetros: {'C': 10, 'gamma': 1}


In [ ]:
Y = vectorizer.transform(textoTestProcesado)
prediction = model.predict(Y)
predictionSearch = grid_search.predict(Y)

# Naive Bayes

Entrenamiento con NB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


#Conunt vectorizacion
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(textoProcesado)


#Vectorizacion TFIDF

#vectorizer = TfidfVectorizer()
#X = vectorizer.fit_transform(textoProcesado)


#model = MultinomialNB()
model = ComplementNB()
model.fit(X, tag)


ComplementNB(alpha=2, norm=True)

In [ ]:
Y = vectorizer.transform(textoTestProcesado)
prediction = model.predict(Y)

In [ ]:

#dfTest["label"] = prediction

dfTest["label"] = predictionSearch

dfTest.to_csv("/content/drive/MyDrive/Practicas_PLN/Competicion/resultados/SVCRBFC100gammaauto.csv")

# FAST TEXT

In [ ]:
pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227141 sha256=d2d11e8fdafa316400596e5a755b882ed7982ca710f79888d4cd66d482bff53b
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
from posixpath import pathsep
import os
import sys
import pandas as pd
import fasttext
from google.colab import drive
drive.mount('/content/drive')


import fasttext
import pandas as pd
from sklearn.metrics import f1_score

train_data = pd.read_csv('/content/drive/MyDrive/Practicas_PLN/Competicion/datos/FinancES_train_kaggle.csv')  # Asegúrate de cambiar el nombre del archivo según corresponda

data = ["__label__{} {}".format(ta, text) for ta,text in zip(tag, textoProcesado)]

with open('train.txt', 'w') as f:
    f.write('\n'.join(data))

model = fasttext.train_supervised(input='train.txt')

predictions = model.predict(textoTestProcesado)

predicted_labels = [label[0].replace("__label__", "") for label in predictions[0]]

dfTest = pd.read_csv("/content/drive/MyDrive/Practicas_PLN/Competicion/datos/FinancES_test_kaggle.csv", index_col=0)

dfTest.drop(columns="text", inplace=True)


dfTest["label"] = predicted_labels

# Guardar el DataFrame con las etiquetas predichas en un archivo CSV
dfTest.to_csv("/content/drive/MyDrive/Practicas_PLN/Competicion/resultados/fastest.csv")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# BERT

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import torch
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

# Definir una clase para el conjunto de datos
class TextDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=None, test_mode=False):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.test_mode = test_mode

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors='pt'
        )

        if self.test_mode:
            return {
                'input_ids': inputs['input_ids'].flatten(),
                'attention_mask': inputs['attention_mask'].flatten(),
            }
        else:
            label = self.labels[idx]
            return {
                'input_ids': inputs['input_ids'].flatten(),
                'attention_mask': inputs['attention_mask'].flatten(),
                'labels': torch.tensor(label, dtype=torch.long)
            }

tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
model = BertForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-cased', num_labels=3)

train_data = pd.read_csv('/content/drive/MyDrive/Practicas_PLN/Competicion/datos/FinancES_train_kaggle.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Practicas_PLN/Competicion/datos/FinancES_test_kaggle.csv')

train_texts = train_data['text'].tolist()
train_labels = train_data['label'].tolist()

max_len = 8
train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_len)

batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

num_epochs = 3
learning_rate = 3e-5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

if test_data is not None:
    test_texts = test_data['text'].tolist()
    test_dataset = TextDataset(test_texts, tokenizer=tokenizer, max_len=max_len, test_mode=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model.eval()
    all_preds = []
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())

dfTest = pd.read_csv("/content/drive/MyDrive/Practicas_PLN/Competicion/datos/FinancES_test_kaggle.csv", index_col=0)
dfTest.drop(columns="text", inplace=True)
dfTest["label"] = all_preds
dfTest.to_csv("/content/drive/MyDrive/Practicas_PLN/Competicion/resultados/BERT3EPOCH.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
